# Text Color Accessibility Classifier

Just a simple hello world like ml program to train a model and predict whether white or black text will look better on given RGB color value.

This uses LinearClassifier, but there is also code for DNNClassifier.

In [0]:
%tensorflow_version 2.x

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as pl
import IPython

Reading csv data file and providing column names

In [101]:
COL_NAMES = ['Red', 'Green', 'Blue', 'Text']
TEXT_COLORS = ['Black', 'White']
df = pd.read_csv('https://raw.githubusercontent.com/kubre/ML-Practice/master/data/color.csv, names= COL_NAMES, header= 0)
# r g b - red green blue
# t - Text color 0 Black, 1 White

SyntaxError: ignored

In [102]:
print(df.head())
print(df.shape)

   Red  Green  Blue  Text
0    0      0     0     1
1    0      0    51     1
2    0      0   102     1
3    0      0   153     1
4    0      0   204     1
(216, 4)


Splitting dataset into two for taining and testing,
you can change frac= x, as ex. x = 0.80 then training dataset
will be 80% and testing dataset will be 20%

In [0]:
df_train = df.sample(frac= 0.80, random_state= 0)
df_test = df.drop(df_train.index)

Extract the labels column

In [0]:
train_y = df_train.pop('Text')
test_y = df_test.pop('Text')

Input function for training and testing purposes.

In [0]:
def input_fn(features, labels, training= True, batch_size= 256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

In [0]:
feature_cols = []

for key in df_train.keys():
  feature_cols.append(tf.feature_column.numeric_column(key= key))

Creating and Training classifier. You can also switch it out for DNNClassifier but in my test it was less accurate than LinearClassifier.
There are lines commented, you can uncomment and see DNNClassifier results.

In [0]:
cl = tf.estimator.LinearClassifier(
    feature_columns= feature_cols,
    n_classes=2
)
cl.train(
    input_fn= lambda: input_fn(df_train, train_y, training= True),
    steps= 8000
)

# DNNClassifer
# cldnn = tf.estimator.DNNClassifier(
#     feature_columns= feature_cols,
#     hidden_units=[20, 10]
#     n_classes=2
# )
# cldnn.train(
#     input_fn= lambda: input_fn(df_train, train_y, training= True),
#     steps= 8000
# )
IPython.display.clear_output()

In [108]:
result = cl.evaluate(input_fn= lambda: input_fn(df_test, test_y, training= False))
# result_dnn = cldnn.evaluate(input_fn= lambda: input_fn(df_test, test_y, training= False))
IPython.display.clear_output()
print('Test set Accuracy: {accuracy:0.3f}'.format(**result))
# print('\nTest set Accuracy(DNN): {accuracy:0.3f}'.format(**result_dnn))

Test set Accuracy: 0.837


In [0]:
# Simple input fucntion for feeding classifier data to predict
def input_fn_p(features, batch_size= 256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

Simple interactive exmaple. 

In [110]:
r_val = 0 #@param { type: 'slider', min: 0, max: 255 }
g_val = 0 #@param { type: 'slider', min: 0, max: 255 }
b_val = 0 #@param { type: 'slider', min: 0, max: 255 }

predict = {
    'Red': [float(r_val)],
    'Green': [float(g_val)],
    'Blue': [float(b_val)]
}

predictions = cl.predict(input_fn= lambda: input_fn_p(predict))

txt_color = ""

for prediction in predictions:
  class_ids = prediction['class_ids'][0]
  probability = prediction['probabilities'][class_ids]
  txt_color = TEXT_COLORS[class_ids]
  IPython.display.clear_output()
  print(f'Text colour should be {TEXT_COLORS[class_ids]}: {probability * 100:.1f}%\nSample: ')

js_code = '''
var div = document.createElement("div");
div.style.width = "100px";
div.style.height = "30px";
div.style.background = "rgb({}, {}, {})";
div.style.paddingTop = "15px";
div.style.textAlign = "center";
div.style.color = "{}";
div.innerHTML = "TEXT";
document.querySelector("#output-area").appendChild(div);
'''.format(r_val, g_val, b_val, txt_color)
display(IPython.display.Javascript(js_code))

Text colour should be White: 99.9%
Sample: 


<IPython.core.display.Javascript object>